In [3]:
import pandas as pd
import numpy as np
import os
import re
import datetime
import pickle

os.chdir("/home/hwiberg/research/COVID_risk/covid19_calculator/calculator/")

import analyzer.loaders.hartford.utils as u
path = '/nfs/sloanlab001/data/HartfordHealthCare/HHCtoMIT/Risk_Calculator/'

file_list = os.listdir(path)

# Find relevant files
file_test = sorted(filter(lambda x: "DYNIdeas_LOS_COVID_Diag_and_Orders" in x, file_list))[-1]
file_add = sorted(filter(lambda x: "hashDYNIdeas_COVID_Prediction_Response_append" in x, file_list))[-1]

### Load test data and filter to inpatient positive cases

In [62]:
version = "inpatient"

In [63]:
df_test = pd.read_csv(path+file_test, sep='|', encoding= 'unicode_escape', dtype = 'object')

In [64]:
df_test.rename(columns={'HOSP_DISCHRG_TIME': 'HOSP_DISCH_TIME'}, inplace = True)

df_test = df_test.assign(
    LAB_RESULT_CLEAN = lambda dataframe: dataframe['LAB_RESULT'].map(lambda x: u.clean_labs(x)),
    PAT_CLASS_CLEAN = lambda dataframe: dataframe['PAT_CLASS'].map(lambda x: u.clean_patient_class(x)),
    MORTALITY_CLEAN = lambda dataframe: dataframe['DISCH_DISP'].map(lambda x: u.clean_mortality(x)),
);

In [65]:
# pd.crosstab(index=df_test['PAT_CLASS'], columns=df_test['PAT_CLASS_CLEAN'])

In [66]:
# Only include positive test results and inpatient visits
df_covid_inpatient = df_test.query('LAB_RESULT_CLEAN == "Positive" & PAT_CLASS_CLEAN == "Inpatient"')

# Only include patients who were discharged (expired patients have discharge times)
df_discharged = df_covid_inpatient.loc[pd.notnull(df_covid_inpatient['HOSP_DISCH_TIME'])]

# Drop duplicate visits
df_discharged = df_discharged[['PAT_MRN_ID', 'PAT_ENC_CSN_ID', 'LOC_PARENT_NAME',
                               'AGE', 'SEX', 'HOSP_ADMSN_TIME', 'HOSP_DISCH_TIME', 'PAT_CLASS_CLEAN', 'MORTALITY_CLEAN']]
df_discharged.drop_duplicates(inplace=True)

# Exclude any patients who appear more than once (unclear what their circumstances were - rare case)
df_discharged.groupby('PAT_MRN_ID').filter(lambda x: len(x) == 1);

In [67]:
df_discharged['HHC_SITE'] = df_discharged['LOC_PARENT_NAME'].map(lambda x: u.hhc_site(x))

pd.crosstab(index = df_discharged['HHC_SITE'], 
            columns = df_discharged['MORTALITY_CLEAN'])

MORTALITY_CLEAN,Alive,Expired
HHC_SITE,,
Main,516,111
Other,329,99


In [68]:
# Load admissions data
df_all = pd.DataFrame()
for file in file_list:
    if file.startswith("hhDYNIdeas_COVID_Prediction_Response_Hist_"):
        print("Reading file: %s" % file)
        df = pd.read_csv(path+file, sep='|', encoding= 'unicode_escape', dtype='object')
        df_all = df_all.append(df)

Reading file: hhDYNIdeas_COVID_Prediction_Response_Hist_03012020_03152020_202005150502.txt
Reading file: hhDYNIdeas_COVID_Prediction_Response_Hist_03162020_03312020_202005150451.txt
Reading file: hhDYNIdeas_COVID_Prediction_Response_Hist_04012020_04152020_202005150351.txt
Reading file: hhDYNIdeas_COVID_Prediction_Response_Hist_04162020_04302020_202005150424.txt
Reading file: hhDYNIdeas_COVID_Prediction_Response_Hist_05012020_05142020_202005150219.txt


## Parse comorbidities

In [69]:
def check_diagnosis(c, dx):
    if pd.isnull(c):
        return False
    else:
        return bool(re.match(".*"+dx, c.lower()))

def diagnosis_filter(dx_list, dx):
    return dx_list[[check_diagnosis(x, dx) for x in dx_list]]

def comorbidity_map(c, dx_list, keyword = None):
    icd_list = list(icd10_map.loc[icd10_map['GROUP_HCUP'] == c,'SHORT_DESCRIPTION'])
    if keyword != None:
        icd_list.extend([keyword])
    all_dx = [diagnosis_filter(dx_list, x.lower()) for x in icd_list]
    dx = list(set().union(*all_dx))
    return dx

In [70]:
df_dx = df_all[['PAT_MRN_ID','COVID_RESEARCH_DX','ENC_DX']].set_index('PAT_MRN_ID')
df_dx_wide = df_dx['COVID_RESEARCH_DX'].str.split("; ",expand=True)
df_dx_wide['ENC_DX'] = df_dx['ENC_DX']
df_dx_long = df_dx_wide.unstack().dropna().reset_index()
df_dx_long.columns = ['level','PAT_MRN_ID','DX']

In [71]:
icd10_map = pd.read_csv("analyzer/hcup_dictionary_icd10.csv")
dx_list = df_dx_long.DX.unique()

COMORB_MAP = {'Essential hypertension': comorbidity_map('Essential hypertension', dx_list, keyword = 'hypertension'),
        'Diabetes':comorbidity_map('Diabetes', dx_list, keyword = 'diabetes'),
        'Coronary atherosclerosis and other heart disease':comorbidity_map('Coronary atherosclerosis and other heart disease', dx_list, keyword = '(atherosclerosis)|(cardiac disease)'),
        'Cardiac dysrhythmias':comorbidity_map('Cardiac dysrhythmias', dx_list, keyword = 'arrhythmia'),
        'Chronic kidney disease':comorbidity_map('Chronic kidney disease', dx_list, keyword = 'chronic (kidney|renal)')
             }

In [84]:
COMORB_MAP['Cardiac dysrhythmias']

['PSVT (paroxysmal supraventricular tachycardia) (HCC)',
 'Palpitations in pediatric patient',
 'Paroxysmal atrial fibrillation with rapid ventricular response (HCC)',
 'Paroxysmal atrial fibrillation (HCC)',
 'Other persistent atrial fibrillation',
 'Recurrent ventricular tachycardia (HCC)',
 'SVT (supraventricular tachycardia) (HCC)',
 'Chronic atrial fibrillation',
 'Palpitations with regular cardiac rhythm',
 'Supraventricular tachycardia (HCC)',
 'Arrhythmia',
 'Paroxysmal supraventricular tachycardia (HCC)',
 'Longstanding persistent atrial fibrillation',
 'Bradycardia, unspecified',
 'VT (ventricular tachycardia) (HCC)',
 'Palpitations',
 'Intermittent palpitations',
 'Ventricular tachycardia (HCC)',
 'Ventricular arrhythmia',
 'Persistent atrial fibrillation',
 'Rapid palpitations',
 'Fetal arrhythmia affecting pregnancy, antepartum',
 'Ventricular tachyarrhythmia (HCC)',
 'Heart palpitations',
 'Tachyarrhythmia',
 'Fetal arrhythmia before the onset of labor',
 'AF (paroxysmal 

In [72]:
for c in u.COMORBIDITIES_COLUMNS:
    df_dx_long[c] = [1 if x in COMORB_MAP[c] else 0 for x in df_dx_long['DX']]

KeyboardInterrupt: 

In [73]:
df_comorb = df_dx_long.drop(['level','DX'], axis = 1)
df_comorb = df_comorb.groupby('PAT_MRN_ID').max().reset_index()

In [ ]:
df_comorb.sum(axis=0)

In [74]:
# Get additional data fields
df_add = pd.read_csv(path+file_add, sep='|', encoding= 'unicode_escape', dtype='object')

## Join into main dataframe
df_joined = df_all.merge(df_add, 
                         on = ['PAT_MRN_ID', 'PAT_ENC_CSN_ID','CALENDAR_DT_STR','HOSP_ADMSN_TIME', 'HOSP_DISCH_TIME',], 
                         how = 'left')
df_joined['ADMISSION_DATE'] = df_joined.HOSP_ADMSN_TIME.apply(u.get_lab_dates)
df_joined['DISCHARGE_DATE'] = df_joined.HOSP_DISCH_TIME.apply(u.get_lab_dates)
df_joined['RECORD_DATE'] = df_joined.CALENDAR_DT_STR.apply(u.get_lab_dates)

In [75]:
## Filter to only first two days and then select first non-null value for each patient/encounter
df_admission = df_joined.loc[df_joined['RECORD_DATE'] <= (df_joined['ADMISSION_DATE'] + datetime.timedelta(days=1)),:]
df_first = df_admission.groupby(['PAT_MRN_ID', 'PAT_ENC_CSN_ID','HOSP_ADMSN_TIME', 'HOSP_DISCH_TIME']).first().reset_index()

In [76]:
# Sanity check
# df_admission.query('PAT_MRN_ID == "847766"').iloc[:,75:90]
# df_first.query('PAT_MRN_ID == "847766"').iloc[:,75:90]

In [77]:
df_withlabs = df_discharged.merge(df_first,
                              on = ['PAT_MRN_ID','PAT_ENC_CSN_ID', 'HOSP_ADMSN_TIME','HOSP_DISCH_TIME'],
                              how = 'inner')
df_full = df_withlabs.merge(df_comorb, on = ['PAT_MRN_ID'], how = 'left');
df_full.rename(columns=u.RENAMED_COLUMNS, inplace=True)

In [78]:
df_full.columns

Index(['PATIENT_ID', 'ENCOUNTER_ID', 'LOC_PARENT_NAME', 'Age', 'Gender',
       'Date_Admission', 'Date_Discharge', 'Patient_Class', 'Outcome',
       'HHC_SITE',
       ...
       'CBC: Leukocytes', 'WHITE_BLOOD_CELL_range', 'ADMISSION_DATE',
       'DISCHARGE_DATE', 'RECORD_DATE', 'Essential hypertension', 'Diabetes',
       'Coronary atherosclerosis and other heart disease',
       'Cardiac dysrhythmias', 'Chronic kidney disease'],
      dtype='object', length=104)

In [79]:
df_full[u.COMORBIDITIES_COLUMNS].sum(axis = 0)

Essential hypertension                              155
Diabetes                                            100
Coronary atherosclerosis and other heart disease      0
Cardiac dysrhythmias                                  1
Chronic kidney disease                               17
dtype: int64

In [80]:
discharge_data = True
comorbidities_data = True
vitals_data = True
lab_tests = True
demographics_data = True
swabs_data = False

cols_select = ['PATIENT_ID', 'ENCOUNTER_ID', 'Date_Admission', 'Date_Discharge', 'Patient_Class', 'HHC_SITE']
if demographics_data:
    cols_select.extend(u.DEMOGRAPHICS_COLUMNS)
if discharge_data:
    cols_select.extend(u.ADMISSIONS_COLUMNS)
if vitals_data:
    cols_select.extend(u.VITALS_COLUMNS)
if lab_tests:
    cols_select.extend(u.LAB_COLUMNS)
if comorbidities_data:
    cols_select.extend(u.COMORBIDITIES_COLUMNS)

In [81]:
## Clean up columns
df_clean = df_full.copy()

df_clean['Glycemia'] = df_clean['Glycemia_1'].combine_first(df_clean['Glycemia_2'])
df_clean.drop(['Glycemia_1', 'Glycemia_2'], axis=1, inplace = True)

## Filter to relevant columns 
df_clean = df_clean.loc[:,cols_select]

# # Parse floats
for lab in u.VITALS_COLUMNS+u.LAB_COLUMNS:
    print(lab)
    df_clean.loc[:,lab] = df_clean.loc[:,lab].apply(u.try_parse_float)
    

## Recode male and female
df_clean.replace({'Male': 0, 'Female': 1}, inplace = True)

df_clean.loc[:,'C-Reactive Protein (CRP)'] = 10*df_clean.loc[:,'C-Reactive Protein (CRP)']

df_clean.loc[:,'Outcome'] = df_clean.loc[:,'Outcome'].map(lambda  x: 1 if x == 'Expired' else 0)

SaO2
Body Temperature
Respiratory Frequency
Systolic Blood Pressure
Cardiac Frequency
ABG: PaO2
Specimen clotted.  Test not performed.
ABG: pH
Specimen clotted.  Test not performed.
Aspartate Aminotransferase (AST)
Specimen hemolyzed. Test not performed.
Specimen hemolyzed. Test not performed.
Specimen hemolyzed. Test not performed.
Hemolyzed specimen, test cannot be performed. Recollection of another specimen is recommended.
Specimen hemolyzed. Test not performed.
Specimen hemolyzed. Test not performed.
Blood Calcium
CBC: Hemoglobin
No specimen received.
CBC: Mean Corpuscular Volume (MCV)
No specimen received.
ABG: MetHb
Specimen clotted.  Test not performed.
C-Reactive Protein (CRP)
Grossly Hemolyzed,Recollection Requested
Test not performed due to insufficient quantity of specimen.
ABG: Lactic Acid
ABG: PaCO2
Specimen clotted.  Test not performed.
Alanine Aminotransferase (ALT)
<5
<5
Hemolyzed specimen, test cannot be performed. Recollection of another specimen is recommended.
<5
<5

In [82]:
df_clean.Outcome.value_counts()

0    459
1     95
Name: Outcome, dtype: int64

In [30]:
## Update
df_clean_main = df_clean.query('HHC_SITE == "Main"')
df_clean_other = df_clean.query('HHC_SITE == "Other"')
df_clean.to_csv("/nfs/sloanlab003/projects/cov19_calc_proj/hartford/hhc_"+version+"_all.csv", index = False)
df_clean_main.to_csv("/nfs/sloanlab003/projects/cov19_calc_proj/hartford/hhc_"+version+"_main.csv", index = False)
df_clean_other.to_csv("/nfs/sloanlab003/projects/cov19_calc_proj/hartford/hhc_"+version+"_other.csv", index = False)
np.transpose(df_clean.describe(include='all')).to_csv("/home/hwiberg/research/COVID_risk/covid19_hartford/descriptive_"+version+".csv", index = True)

In [28]:
df_clean.columns

Index(['PATIENT_ID', 'ENCOUNTER_ID', 'Date_Admission', 'Date_Discharge',
       'Patient_Class', 'HHC_SITE', 'Gender', 'Age', 'Outcome', 'SaO2',
       'Body Temperature', 'Respiratory Frequency', 'Systolic Blood Pressure',
       'Cardiac Frequency', 'ABG: PaO2', 'ABG: pH',
       'Aspartate Aminotransferase (AST)', 'Blood Calcium', 'CBC: Hemoglobin',
       'CBC: Mean Corpuscular Volume (MCV)', 'ABG: MetHb',
       'C-Reactive Protein (CRP)', 'ABG: Lactic Acid', 'ABG: PaCO2',
       'Alanine Aminotransferase (ALT)', 'Blood Amylase', 'Blood Creatinine',
       'Blood Sodium', 'Potassium Blood Level', 'ABG: COHb', 'D-Dimer',
       'CBC: Platelets', 'Total Bilirubin',
       'CBC: Red cell Distribution Width (RDW)', 'Cholinesterase', 'Glycemia',
       'Prothrombin Time (INR)', 'Blood Urea Nitrogen (BUN)',
       'CBC: Leukocytes', 'Essential hypertension', 'Diabetes',
       'Coronary atherosclerosis and other heart disease',
       'Cardiac dysrhythmias', 'Chronic kidney disease'],
 

In [29]:
pd.crosstab(index = df_clean['Patient_Class'], 
            columns = df_clean['Outcome'])

Outcome,0,1
Patient_Class,,
Inpatient,459,95
